# Google Trend
After the three topics for each article was identied, the goal was to create a metric for how relevant was this topic at the date the article was published? This done under the assumption that people read articles that are relevant for the current time. If the topic is not 'popular' at the time of release, it will not be well resieved by the end-users, hence the user engangement would decrease. <br>
An example of this could be the topic of Covid-19. If you had written and article about Covid-19 back in Febrauary 2020, chanches are that most people would be very interested in that article. Whereas today, nobody wants to read anymore about covid-19.  <br>
This next section will provide each topic with a metric of how relevant this topic is at the time of release. 

In order to determine a topics relevance `pytrend` is used, which is an unofficial API for Google Trends. Google Trend is a great tool for mapping what poeple are seaching for, in real time. 

In [3]:
# Install pytrends
# !pip install pytrends

In [4]:
import pandas as pd                        
import time
import itertools
from pytrends.request import TrendReq
pytrends = TrendReq()

# Delete these
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
pd.set_option('display.max_rows', 10)

Mounted at /content/drive


In [5]:
data_path = "/content/drive/MyDrive/DTU/"
df = pd.read_csv(data_path + "data_topic.csv")

# Nessesary date preprocessing step
df['date'] = df['published_at'].str.split('T', expand=True)[0]
df['date'] = df.date.str.rsplit('-', 1, expand=True)[0]
# df = df.iloc[[0,-2]]
df = df.head(2).append(df.tail(2))
df

,Unnamed: 0,Unnamed: 0.1,source_id,source_name,author,title,description,url,url_to_image,published_at,...,cleaned_content,stop_removed_content,tokenized,lemmatized,lemmatize_joined,Topic,Topic1,Topic2,Topic3,date
0,0,0,reuters,Reuters,Reuters Editorial,NTSB says Autopilot engaged in 2018 California...,The National Transportation Safety Board said ...,https://www.reuters.com/article/us-tesla-crash...,https://s4.reutersmedia.net/resources/r/?m=02&...,2019-09-03T16:22:20Z,...,the national transportation safety board said ...,national transportation safety board said tues...,"['national', 'transportation', 'safety', 'boar...","['national', 'transportation', 'safety', 'boar...",national transportation safety board say tuesd...,5,crash,plane,flight,2019-09
1,1,1,the-irish-times,The Irish Times,Eoin Burke-Kennedy,Unemployment falls to post-crash low of 5.2%,Latest monthly figures reflect continued growt...,https://www.irishtimes.com/business/economy/un...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T10:32:28Z,...,the states jobless rate fell to per cent last ...,states jobless rate fell per cent last month a...,"['states', 'jobless', 'rate', 'fell', 'per', '...","['state', 'jobless', 'rate', 'fell', 'per', 'c...",state jobless rate fell per cent last month ac...,-1,NaN,NaN,NaN,2019-09
8118,8118,10432,abc-news,ABC News,The Associated Press,Drop in US service sector activity raises econ...,"Get breaking national and world news, broadcas...",https://abcnews.go.com/Business/wireStory/drop...,https://s.abcnews.com/images/US/WireAP_eb147c9...,2019-10-03T16:30:16Z,...,growth in the economys vast services sector ...,growth economys vast services sector slowed sh...,"['growth', 'economys', 'vast', 'services', 'se...","['growth', 'economys', 'vast', 'service', 'sec...",growth economys vast service sector slow sharp...,4,trade,china,stock,2019-10
8119,8119,10433,reuters,Reuters,Sumeet Chatterjee,Banker defections pose challenge for Credit Su...,The announcement by Julius Baer this week that...,https://www.reuters.com/article/us-credit-suis...,https://s3.reutersmedia.net/resources/r/?m=02&...,2019-10-03T15:59:52Z,...,the announcement by julius baer this week that...,announcement julius baer week hired five senio...,"['announcement', 'julius', 'baer', 'week', 'hi...","['announcement', 'julius', 'baer', 'week', 'hi...",announcement julius baer week hire five senior...,84,suisse,credit,ag,2019-10


I order to optimize the computional power the three columns with topics are combined and the `unique()` function is used to find every unique topic. This limits the number of requests made to Google Trends API, seeing that many of the topics are seen in more than one topic column. <br>

`pytrend` is in many ways an easy and great tool, but is comes with many limitations. The timeframe of which a topic is investigates can not surpass more than 10 years from today. Luckily the lastest date in this data set is from September 2019. The date format is also very restrictive, as it only works for each seventh day in the month. A decision was made to shorten the date format, which originaly was in `YYYY-MM-DD` to simply a `YYYY-MM` format. This reduced the number of dates to two dates (2019-09 and 2019-10). That is why the input of the function is only a single date string. <br>

A final remark about the function was that the topic = "date" would not be accepted as input in `pytrends`, which is why it simply was just removed. 

In [11]:
## September 2019
df_09 = df[df['date']=='2019-09']
topic_09_list = []
topic_09_list.append(df_09['Topic1'].unique())
topic_09_list.append(df_09['Topic2'].unique())
topic_09_list.append(df_09['Topic3'].unique())

topic_09_list = list(itertools.chain.from_iterable(topic_09_list))
topic_09_list = list(set(topic_09_list))
# topic_09_list.remove('date')

## October 2019
df_10 = df[df['date']=='2019-10']
topic_10_list = []
topic_10_list.append(df_10['Topic2'].unique())
topic_10_list.append(df_10['Topic3'].unique())
topic_10_list.append(df_10['Topic1'].unique())

topic_10_list = list(itertools.chain.from_iterable(topic_10_list))
topic_10_list = list(set(topic_10_list))
# topic_10_list.remove('date')

When is comes the actual value returned by API, `pytrends`agian comes with its limitations. According to Lazarina Stoy from October 2021 she says the following about the returned value:

> Values are calculated on a scale from 0 to 100, where 100 is the location with the most popularity as a fraction of total searches in that location, a value of 50 indicates a location that is half as popular, and a value of 0 indicates a location where the term was less than 1% as popular as the peak. (Soruce: https://lazarinastoy.com/the-ultimate-guide-to-pytrends-google-trends-api-with-python/)

The following function looks at a time period from five years ago to today. Depending on the month and year selected it returns a mean value for the topic for that specific month. It is averaged because `pytrends` still returns the every seven day of the month. It is assumed in this project that the interest of a topic is constant throughout a month. 

In [12]:
def Topic_Value(date,topic):
    # Initialize pytrends API request
    pytrends.build_payload([topic], cat=0, timeframe='today 5-y') 
    data = pytrends.interest_over_time() 
    data = data.reset_index() 
    
    # Group to only see year and month
    data['YearMonth'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m')
    # Average for the enitre month
    data = data.groupby('YearMonth').mean()
    # Find value for topic 
    value = data[topic].filter(items = [date], axis=0)[0]

    # Should be increased if not runned in Google Colab.
    time.sleep(3)
    return value

The next code chunk then takes the list of topics and use the `Topic_Value` function to pull a value and saves it in a list with the coresponding values for that topic. 

In [13]:
topic_value_oct = []
topic_value_sep = []

# Topics and values from September 2019 
for i in topic_09_list:
  topic_value_sep.append(Topic_Value('2019-09',i))
  # print(i,';', topic_value_sep[-1])

# Topics and values from October 2019 
for i in topic_10_list:
  topic_value_oct.append(Topic_Value('2019-10',i))
  # print(i,';', topic_value_oct[-1])

nan ; 70.4
crash ; 21.2
plane ; 47.8
flight ; 86.2
ag ; 88.0
stock ; 21.5
credit ; 88.5
suisse ; 15.75
china ; 35.75
trade ; 67.0


The next code chunks are made to combine the topic and values for the different timestamps the dataframe.

In [17]:
df_topics_1 = pd.DataFrame({'date': '2019-09','Topic1': topic_09_list,'Topic 1 Score': topic_value_sep})
df_topics_2 = pd.DataFrame({'date': '2019-09','Topic2': topic_09_list,'Topic 2 Score': topic_value_sep})
df_topics_3 = pd.DataFrame({'date': '2019-09','Topic3': topic_09_list,'Topic 3 Score': topic_value_sep})

df_topics_1_10 = pd.DataFrame({'date': '2019-10','Topic1': topic_10_list,'Topic 1 Score': topic_value_oct})
df_topics_2_10 = pd.DataFrame({'date': '2019-10','Topic2': topic_10_list,'Topic 2 Score': topic_value_oct})
df_topics_3_10 = pd.DataFrame({'date': '2019-10','Topic3': topic_10_list,'Topic 3 Score': topic_value_oct})

df_topics_1 = df_topics_1.append(df_topics_1_10)
df_topics_2 = df_topics_2.append(df_topics_2_10)
df_topics_3 = df_topics_3.append(df_topics_3_10)

df = pd.merge(df, df_topics_1,  how='left', left_on=['date','Topic1'], right_on = ['date','Topic1'])
df = pd.merge(df, df_topics_2,  how='left', left_on=['date','Topic2'], right_on = ['date','Topic2'])
df = pd.merge(df, df_topics_3,  how='left', left_on=['date','Topic3'], right_on = ['date','Topic3'])

In [ ]:
df.to_csv(data_path + 'dataset.csv')